<a href="https://colab.research.google.com/github/tasjapr/diploma/blob/master/my_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preset

In [0]:
#check GPU

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
!git clone https://github.com/tensorflow/models.git

In [0]:
#connect to google drive

from google.colab import drive
drive.mount('/content/drive')

In [0]:
# copy necessary files from google drive

!cp -r drive/My\ Drive/Colab\ Notebooks/files/ /content/

In [0]:
import os
os.chdir('models/')
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
    
os.chdir('models/research/')

!protoc object_detection/protos/*.proto --python_out=.

#TensorFlow object detection API install

https://colab.research.google.com/drive/1dU0bKSzYt8mz83lsI09tMhn-A7w1Y9MY#scrollTo=CWuJDoKJHMSs&forceEdit=true&offline=true&sandboxMode=true

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib

In [0]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

In [0]:
import os
os.chdir('/content/models/research')

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
!protoc object_detection/protos/*.proto --python_out=.
!pip install 'prompt-toolkit==1.0.15'

In [0]:
# check installation

!python object_detection/builders/model_builder_test.py

# generate tf_record

In [0]:
# !cp /content/drive/My\ Drive/Colab\ Notebooks/files/images/train /content/files/images/

In [0]:
# !cp -r /content/drive/My\ Drive/Colab\ Notebooks/files/images/train /content/files/images/

In [0]:
# !cp -r /content/drive/My\ Drive/Colab\ Notebooks/files/images/test/s001-00_img.xml /content/files/images/test/
# !cp -r /content/drive/My\ Drive/Colab\ Notebooks/files/images/test/s006-02_img.xml /content/files/images/test/
# !cp -r /content/drive/My\ Drive/Colab\ Notebooks/files/images/test/s014-06_img.xml /content/files/images/test/
# !cp -r /content/drive/My\ Drive/Colab\ Notebooks/files/images/test/s021-01_img.xml /content/files/images/test/

# import os
# os.chdir('../../files/')

In [0]:
#!cp /content/files/generate_tfrecord.py /content/models/research/

In [0]:
!python3 xml_to_csv.py

In [0]:
!python3 generate_tfrecord.py --csv_input=/content/files/data/train_labels.csv --output_path=/content/files/data/train.record --image_dir=/content/files/images/train/
!python3 generate_tfrecord.py --csv_input=/content/files/data/test_labels.csv --output_path=/content/files/data/test.record --image_dir=/content/files/images/test/

# Training model

In [0]:
!cp -r xml_to_csv/data/ models/research/object_detection/
!cp -r xml_to_csv/images/ models/research/object_detection/

In [0]:
import os
os.chdir('models/research/object_detection/')

In [0]:
!mkdir training

In [0]:
import os
os.chdir('../../../xml_to_csv/')

In [0]:
!cp ssd_mobilenet_v2_coco.config ../models/research/object_detection/training/

In [0]:
!mv ../../../../ssd_mobilenet_v2_coco.config ../training/

In [0]:
!mv ssd_mobilenet_v2_coco.config models/research/object_detection/training

In [0]:
!python3 train.py --logtostderr --train_dir=../training/ --pipeline_config_path=../training/ssd_mobilenet_v2_coco.config